# Basic RAG
Retrieval-augmented generation (RAG) is an AI framework that synergizes the capabilities of LLMs and information retrieval systems. It’s useful to answer questions or generate content leveraging external knowledge. There are two main steps in RAG: 1) retrieval: retrieve relevant information from a knowledge base with text embeddings stored in a vector store; 2) generation: insert the relevant information to the prompt for the LLM to generate information. In this file, we will build a basic example of RAG with four implementations:

- RAG from scratch with Mistral
- RAG with Mistral and LangChain
- RAG with Mistral and LlamaIndex

### Import needed packages
The first step is to install the needed packages `mistralai` and `faiss-cpu` and import the needed packages:



In [1]:
! pip3 install faiss-cpu==1.7.4 mistralai

  Using cached faiss-cpu-1.7.4.tar.gz (57 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for faiss-cpu (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      running bdist_wheel
      running build
      running build_py
      running build_ext
      building 'faiss._swigfaiss' extension
      swigging faiss/faiss/python/swigfaiss.i to faiss/faiss/python/swigfaiss_wrap.cpp
      swig -python -c++ -Doverride= -I/usr/local/include -Ifaiss -doxygen -module swigfaiss -o faiss/faiss/python/swigfaiss_wrap.cpp faiss/faiss/python/swigfaiss.i
      error: command 'swig' failed: No such file or directory
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for faiss-cpu
Failed to build faiss-cpu
ERROR: ERROR: Failed to bu

In [2]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import requests
import numpy as np
import faiss
import os
from getpass import getpass

api_key= os.getenv("mistral_api_key")
client = MistralClient(api_key=api_key)

ModuleNotFoundError: No module named 'mistralai'

### Get data

In this very simple example, we are getting data from the late payment policy.

In [33]:
file_path='Late-Payment-Policy_FM1.txt'
def read_text_file(file_path):
    with open(file_path, 'r') as file:
        text_content = file.read()
    return text_content
text = read_text_file(file_path)
print(text)

 
Late Payment Policy (FM1) 
 
1 
July 2019 version 
 
The University of British Columbia  
Board of Governors 
 
 
Policy No.: 
FM1 
Long Title: 
Late Payment of Fees and Accounts 
 
Short Title:  
Late Payment Policy 
 
 
 
1. General 
 
1.1 
Where  fees,  fines,  or  other  indebtedness  to  the  University  remain  unpaid  despite  the 
University having taken reasonable steps to notify the individual concerned, the University 
may report the outstanding obligation to credit reporting agencies, commence legal action, 
or utilize any other remedies that may be available to it, whether the outstanding obligation 
is owed by a faculty member, staff member, student, or other individual. 
 
1.2 
A late payment fee and interest may be charged. 
 
1.3 
In cases where the outstanding obligation is owed by a student, the University will attempt to 
secure payment using internal processes prior to commencing any legal action. Provided that 
the  University  has  first  taken  reasonable  ste

In [34]:
file_path2='Study-Leave-Policy_HR8.txt'
def read_text_file(file_path):
    with open(file_path, 'r') as file:
        text_content = file.read()
    return text_content
text2 = read_text_file(file_path2)
print(text2)

 
Study Leave Policy (HR8) 
 
1 
July 2019 version 
 
The University of British Columbia  
Board of Governors 
 
 
Policy No.: 
HR8 
Long Title: 
Study Leave (Other Than Faculty) 
 
Short Title:  
Study Leave Policy 
 
 
 
1. General 
 
1.1 
Heads  and  Directors  of  Academic  Service  Departments  and  of  Administrative  Service 
Departments, and the Associate and Assistant Heads or Directors of these Departments, as well 
as other professional staff as approved from time to time by the President, are eligible to apply 
for a study leave program. The prescribed conditions are as follows: 
 
1.1.1 
Eligibility ‐ After four years (4) continuous service. 
 
1.1.2 
Entitlement ‐ Three (3) plus the number of years of full‐time service equals the number 
of months of partially paid study leave to a maximum of one year. 
 
2. Study Leave Salary 
 
2.1 
Fifty  percent  (50%)  of  basic  salary  together  with  the  University’s  full  contribution  to  fringe 
benefits, provided the individ

In [35]:
len(text)
len(text2)

3747

## Split document into chunks

In a RAG system, it is crucial to split the document into smaller chunks so that it’s more effective to identify and retrieve the most relevant information in the retrieval process later. In this example, we simply split our text by character, combine 4765 characters into each chunk, and we get 10 chunks.

In [36]:
chunk_size = 512
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

In [37]:
chunk_size = 512
chunks = [text2[i:i + chunk_size] for i in range(0, len(text2), chunk_size)]

In [38]:
len(chunks)

8

#### Considerations:
- **Chunk size**: Depending on your specific use case, it may be necessary to customize or experiment with different chunk sizes and chunk overlap to achieve optimal performance in RAG. For example, smaller chunks can be more beneficial in retrieval processes, as larger text chunks often contain filler text that can obscure the semantic representation. As such, using smaller text chunks in the retrieval process can enable the RAG system to identify and extract relevant information more effectively and accurately.  However, it’s worth considering the trade-offs that come with using smaller chunks, such as increasing processing time and computational resources.
- **How to split**: While the simplest method is to split the text by character, there are other options depending on the use case and document structure. For example, to avoid exceeding token limits in API calls, it may be necessary to split the text by tokens. To maintain the cohesiveness of the chunks, it can be useful to split the text by sentences, paragraphs, or HTML headers. If working with code, it’s often recommended to split by meaningful code chunks for example using an Abstract Syntax Tree (AST) parser.


### Create embeddings for each text chunk
For each text chunk, we then need to create text embeddings, which are numeric representations of the text in the vector space. Words with similar meanings are expected to be in closer proximity or have a shorter distance in the vector space.
To create an embedding, use Mistral’s embeddings API endpoint and the embedding model `mistral-embed`. We create a `get_text_embedding` to get the embedding from a single text chunk and then we use list comprehension to get text embeddings for all text chunks.


In [39]:
def get_text_embedding(input):
    embeddings_batch_response = client.embeddings(
          model="mistral-embed",
          input=input
      )
    return embeddings_batch_response.data[0].embedding

In [40]:
text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

MistralAPIException: Status: 403. Message: {"message":"Inactive subscription or usage limit reached"}

In [ ]:
text_embeddings.shape

(8, 1024)

In [ ]:
text_embeddings

array([[-0.06896973,  0.03579712,  0.04455566, ..., -0.01472473,
         0.00475311,  0.01698303],
       [-0.05960083,  0.03814697,  0.02752686, ..., -0.002388  ,
         0.01235199,  0.02053833],
       [-0.06817627,  0.03305054,  0.03308105, ..., -0.00751495,
        -0.01160431,  0.01058197],
       ...,
       [-0.07092285,  0.02958679,  0.04815674, ..., -0.01805115,
         0.00206375,  0.01121521],
       [-0.05944824,  0.02044678,  0.0317688 , ..., -0.00389099,
         0.01121521, -0.00862885],
       [-0.0475769 ,  0.02705383,  0.03640747, ..., -0.02383423,
        -0.00164032, -0.00032401]])

### Load into a vector database
Once we get the text embeddings, a common practice is to store them in a vector database for efficient processing and retrieval. There are several vector database to choose from. In our simple example, we are using an open-source vector database Faiss, which allows for efficient similarity search.  

With Faiss, we instantiate an instance of the Index class, which defines the indexing structure of the vector database. We then add the text embeddings to this indexing structure.


In [ ]:
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

#### Considerations:
- **Vector database**: When selecting a vector database, there are several factors to consider including speed, scalability, cloud management, advanced filtering, and open-source vs. closed-source.

### Create embeddings for a question
Whenever users ask a question, we also need to create embeddings for this question using the same embedding models as before.


In [ ]:
question = "What may the University charge when fees, fines, or other indebtedness remain unpaid?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape

(1, 1024)

In [ ]:
question_embeddings

array([[-0.02053833,  0.0625    ,  0.04031372, ..., -0.01794434,
        -0.00689316, -0.00888824]])

#### Considerations:
- Hypothetical Document Embeddings (HyDE): In some cases, the user’s question might not be the most relevant query to use for identifying the relevant context. Instead, it maybe more effective to generate a hypothetical answer or a hypothetical document based on the user’s query and use the embeddings of the generated text to retrieve similar text chunks.

### Retrieve similar chunks from the vector database
We can perform a search on the vector database with `index.search`, which takes two arguments: the first is the vector of the question embeddings, and the second is the number of similar vectors to retrieve. This function returns the distances and the indices of the most similar vectors to the question vector in the vector database. Then based on the returned indices, we can retrieve the actual relevant text chunks that correspond to those indices.


In [ ]:
D, I = index.search(question_embeddings, k=2)
print(I)

[[1 5]]


In [ ]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

['\xa0legal\xa0action,\xa0\nor\xa0utilize\xa0any\xa0other\xa0remedies\xa0that\xa0may\xa0be\xa0available\xa0to\xa0it,\xa0whether\xa0the\xa0outstanding\xa0obligation\xa0\nis\xa0owed\xa0by\xa0a\xa0faculty\xa0member,\xa0staff\xa0member,\xa0student,\xa0or\xa0other\xa0individual.\xa0\n\xa0\n1.2 \nA\xa0late\xa0payment\xa0fee\xa0and\xa0interest\xa0may\xa0be\xa0charged.\xa0\n\xa0\n1.3 \nIn\xa0cases\xa0where\xa0the\xa0outstanding\xa0obligation\xa0is\xa0owed\xa0by\xa0a\xa0student,\xa0the\xa0University\xa0will\xa0attempt\xa0to\xa0\nsecure\xa0payment\xa0using\xa0internal\xa0processes\xa0prior\xa0to\xa0commencing\xa0any\xa0legal\xa0action.\xa0Provided\xa0that\xa0\nthe\xa0 University\xa0 has\xa0 first\xa0 taken\xa0 reasonable\xa0 steps\xa0 to\xa0 notify\xa0 the\xa0 ind', '\xa0unit\xa0in\xa0which\xa0the\xa0outstanding\xa0obligation\xa0was\xa0incurred\xa0shall\xa0\ntake\xa0reasonable\xa0steps\xa0to\xa0notify\xa0the\xa0individual\xa0concerned\xa0before\xa0taking\xa0any\xa0further\xa0steps.\xa0Such\xa0\n

#### Considerations:
- **Retrieval methods**: There are a lot different retrieval strategies. In our example, we are showing a simple similarity search with embeddings. Sometimes when there is metadata available for the data, it’s better to filter the data based on the metadata first before performing similarity search. There are also other statistical retrieval methods like TF-IDF and BM25 that use frequency and distribution of terms in the document to identify relevant text chunks.
- **Retrieved document**: Do we always retrieve individual text chunk as it is? Not always.
    - Sometimes, we would like to include more context around the actual retrieved text chunk. We call the actual retrieve text chunk “child chunk” and our goal is to retrieve a larger “parent chunk” that the “child chunk” belongs to.
    - On occasion, we might also want to provide weights to our retrieve documents. For example, a time-weighted approach would help us retrieve the most recent document.
    - One common issue in the retrieval process is the “lost in the middle” problem where the information in the middle of a long context gets lost. Our models have tried to mitigate this issue. For example, in the passkey task, our models have demonstrated the ability to find a "needle in a haystack" by retrieving a randomly inserted passkey within a long prompt, up to 32k context length. However, it is worth considering experimenting with reordering the document to determine if placing the most relevant chunks at the beginning and end leads to improved results.
  
### Combine context and question in a prompt and generate response

Finally, we can offer the retrieved text chunks as the context information within the prompt. Here is a prompt template where we can include both the retrieved text and user question in the prompt.



In [ ]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [ ]:
def run_mistral(user_message, model="mistral-medium-latest"):
    messages = [
        ChatMessage(role="user", content=user_message)
    ]
    chat_response = client.chat(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

In [ ]:
run_mistral(prompt)

'According to the context information, if fees, fines, or other indebtedness remain unpaid, the University may charge a late payment fee and interest on the outstanding obligation. Additionally, the University may attempt to secure payment using internal processes prior to commencing legal action. In cases where the obligation is owed by a student, the administrative unit may decline to provide further services to the individual if the outstanding obligation remains unpaid despite attempts at notification. It is important to note that the administrative unit in which the outstanding obligation was incurred shall take reasonable steps to notify the individual concerned before taking any further steps. Such notification shall state the late fees or interest charges, if any, which apply to the outstanding obligation, as well as the potential consequences of non-payment.'

#### Considerations:
- Prompting techniques: Most of the prompting techniques can be used in developing a RAG system as well. For example, we can use few-shot learning to guide the model’s answers by providing a few examples. Additionally, we can explicitly instruct the model to format answers in a certain way.


In the next sections, we are going to show you how to do a similar basic RAG with some of the popular RAG frameworks. We will start with LlamaIndex and add other frameworks in the future.


## LlamaIndex

In [ ]:
!pip3 install llama-index llama-index-llms-mistralai llama-index-embeddings-mistralai


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [ ]:
import os
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding




In [ ]:
# Load data
reader = SimpleDirectoryReader(input_files=["Late-Payment-Policy_FM1.txt"])
documents = reader.load_data()
# Define LLM and embedding model
Settings.llm = MistralAI(model="open-mistral-7b", api_key=api_key)
Settings.embed_model = MistralAIEmbedding(model_name='mistral-embed', api_key=api_key)
# Create vector store index
index = VectorStoreIndex.from_documents(documents)

In [ ]:
# Create query engine
query_engine = index.as_query_engine(similarity_top_k=2)
response = query_engine.query(
    "What may the University charge when fees, fines, or other indebtedness remain unpaid?"
)



In [ ]:
import textwrap

def format_response(response):
    response_text = str(response)
    return '\n'.join(textwrap.wrap(response_text, width=80))  # Wrap text at 80 characters

formatted_response = format_response(response)
print(formatted_response)

The University may charge late fees or interest when fees, fines, or other
indebtedness remain unpaid.


In [ ]:
response2 = query_engine.query(
    "Are individual academic departments authorized to withhold grades from Enrolment Services for any reason?"
)

formatted_response2 = format_response(response2)
print(formatted_response2)

No, individual academic departments are not authorized to withhold grades from
Enrolment Services for any reason, according to the Late-Payment-Policy_FM1.txt.
This is stated in section 1.4 of the July 2019 version of the policy.


In [ ]:

response3 = query_engine.query(
    "What steps must an administrative unit take before declining further services to an individual with outstanding obligations?"
)
formatted_response3 = format_response(response3)
print(formatted_response3)

An administrative unit must take reasonable steps to notify the individual
concerned before declining further services. This notification should state the
late fees or interest charges, if any, which apply to the outstanding
obligation, as well as the potential consequences of non-payment.


In [ ]:

response4 = query_engine.query(
    "What actions may the Department of Housing and Conferences take if a resident has outstanding obligations?")
formatted_response4 = format_response(response4)
print(formatted_response4)

The Department of Housing and Conferences may refuse admission to residences and
may withdraw residence privileges, including dining privileges, requiring a
resident to vacate the premises.


In [ ]:
response5 = query_engine.query(
    "What may Parking and Access Control Services do in cases of unpaid obligations?")
formatted_response5 = format_response(response5)
print(formatted_response5)

Parking and Access Control Services may withdraw parking privileges and may tow
vehicles.


In [ ]:
# Load data
reader_sl = SimpleDirectoryReader(input_files=["Study-Leave-Policy_HR8.txt"])
documents = reader_sl.load_data()
# Define LLM and embedding model
Settings.llm = MistralAI(model="open-mistral-7b", api_key=api_key)
Settings.embed_model = MistralAIEmbedding(model_name='mistral-embed', api_key=api_key)
# Create vector store index
index = VectorStoreIndex.from_documents(documents)

In [ ]:
# Create query engine
query_engine = index.as_query_engine(similarity_top_k=2)
response_sl_1 = query_engine.query(
    "What are the entitlements for study leave?"
)


In [ ]:
formatted_response_sl_1 = format_response(response_sl_1)
print(formatted_response_sl_1)

The entitlement for study leave, according to the provided context, is three
months plus the number of years of full-time service, up to a maximum of one
year. This means that after four years of continuous service, an individual is
entitled to a study leave of up to one year, with three months being the base
and additional months being equal to the number of years of full-time service.
This study leave is partially paid, with the individual receiving 50% of their
basic salary, along with the University's full contribution to fringe benefits,
provided the individual continues their own contributions.


# Langchain + Mistral AI


In [ ]:
!pip3 install langchain_community

In [ ]:
!pip3 install langchain_mistralai

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

import requests
import numpy as np
import faiss
import os
from getpass import getpass

api_key= os.getenv("mistral_api_key")



In [4]:
# Load data
loader = TextLoader("Late-Payment-Policy_FM1.txt")
docs = loader.load()
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
# Define the embedding model
embeddings = MistralAIEmbeddings(model="mistral-embed", mistral_api_key=api_key)
# Create the vector store
vector = FAISS.from_documents(documents, embeddings)
# Define a retriever interface
retriever = vector.as_retriever()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_mistralai/embeddings.py:105: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [5]:
# Define LLM
model = ChatMistralAI(mistral_api_key=api_key)
# Define prompt template
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "What may the University charge when fees, fines, or other indebtedness remain unpaid"})
print(response["answer"])

When fees, fines, or other indebtedness to the University remain unpaid, the University may charge late fees or interest, as stated in the Late Payment Policy (FM1) under section 1.5. The specific amount of late fees or interest that may be charged is not specified in the context provided.


In [7]:
response2 = retrieval_chain.invoke({"input": "What actions may the Department of Housing and Conferences take if a resident has outstanding obligations?"})
print(response2["answer"])

The Department of Housing and Conferences may refuse admission to residences and may withdraw residence privileges, including dining privileges, requiring a resident to vacate the premises if the resident has outstanding obligations.


In [8]:
response2 = retrieval_chain.invoke({"input": "What may Parking and Access Control Services do in cases of unpaid obligations?"})
print(response2["answer"])

Parking and Access Control Services may withdraw parking privileges and may tow vehicles in cases of unpaid obligations.


In [9]:
response3 = retrieval_chain.invoke({"input": "Are individual academic departments authorized to withhold grades from Enrolment Services for any reason?"})
print(response3["answer"])

No, individual academic departments are not authorized to withhold grades from Enrolment Services for any reason. This is stated in section 1.4 of the Late Payment Policy (FM1) from July 2019 version.


In [12]:
response4 = retrieval_chain.invoke({"input": "What actions may the University take if fees, fines, or other indebtedness remain unpaid?"})
print(response4["answer"])

If fees, fines, or other indebtedness to the University remain unpaid despite the University's attempts to notify the individual concerned, the University may take several actions. These can include reporting the outstanding obligation to credit reporting agencies, commencing legal action, or utilizing any other remedies available. The University may also charge a late payment fee and interest.

In cases where the outstanding obligation is owed by a student, the University will attempt to secure payment using internal processes prior to commencing any legal action. These internal processes may include refraining from making additional services or privileges available to the student. Specifically, the University, acting through Enrolment Services, may decline to process an application for admission as a student, allow subsequent registration, or provide academic transcripts or otherwise make grade information available.

However, individual academic departments within the University are

In [13]:
response5 = retrieval_chain.invoke({"input": "What are the internal processes the University may use to secure payment from a student before commencing legal action?"})
print(response5["answer"])

Based on the provided context, the internal processes the University may use to secure payment from a student before commencing legal action include:

1.3.1 Declining to process an application for admission as a student.
1.3.2 Declining to allow subsequent registration.
1.3.3 Declining to provide academic transcripts or otherwise make grade information available.

These steps are taken through Enrolment Services, which manages the student's academic record and related services. The University will attempt to use these internal processes prior to commencing any legal action, provided that it has first taken reasonable steps to notify the individual concerned.


In [18]:
response6 = retrieval_chain.invoke({"input": "What steps must an administrative unit take before declining further services to an individual with outstanding obligations?"})
print(response6["answer"])

According to the provided context, before an administrative unit at the University of British Columbia declines further services to an individual with outstanding obligations, they must take the following steps:

1. Take reasonable steps to notify the individual concerned about the outstanding obligation, stating the late fees or interest charges, if any, as well as the potential consequences of non-payment.
2. If the outstanding obligation remains unpaid despite the attempts at notification, the administrative unit may decline to provide further services to the individual. However, this should not be done without first notifying the individual.

It's important to note that the specific consequences of non-payment may vary depending on the administrative unit. For example, the Department of Housing and Conferences may refuse admission to residences and withdraw residence privileges, Parking and Access Control Services may withdraw parking privileges and tow vehicles, and the Library ma

In [19]:

response7 = retrieval_chain.invoke({"input": "What may the Library do if an individual has outstanding obligations?"})
print(response7["answer"])

According to the provided context, if an individual has outstanding obligations to the University of British Columbia, the Library may withdraw borrowing privileges and access to its collection of electronic information resources. This is outlined in section 1.2.3 of the procedures associated with the Late Payment Policy.


In [21]:

response8 = retrieval_chain.invoke({"input": "What steps must be taken if an administrative unit forwards information about a student’s outstanding obligation to Enrolment Services?"})
print(response8["answer"])

If an administrative unit forwards information about a student’s outstanding obligation to Enrolment Services, the administrative unit must at the same time take reasonable steps to notify the student that until the obligation is paid in full, Enrolment Services will not process an application for admission as a student, allow subsequent registration, or provide academic transcripts or otherwise make grade information available (as per section 1.4 of the July 2002 version of the Procedures to Late Payment Policy (FM1)). Additionally, Enrolment Services may add an administrative fee to the outstanding obligation (as per the context provided).


In [25]:

response9 = retrieval_chain.invoke({"input": "Can Enrolment Services add an administrative fee to an outstanding obligation referred to them by an administrative unit?"})
print(response9["answer"])

Yes, based on the context provided, it is stated that "Where outstanding obligations are referred to Enrolment Services, Enrolment Services may add an administrative fee to the outstanding obligation." This implies that Enrolment Services has the authority to add an administrative fee to an outstanding obligation that has been referred to them by an administrative unit.


# Integrate KG with RAG

### Connecting neo4j

In [27]:
from dotenv import load_dotenv
import os

from langchain_community.graphs import Neo4jGraph

# Warning control
import warnings
warnings.filterwarnings("ignore")

# Late Payment KG

In [28]:

load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')

In [29]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [30]:
print(kg.schema)

Node properties:
Organization {name: STRING}
Role {name: STRING}
Rule {description: STRING, name: STRING}
Consequence {name: STRING}
Relationship properties:

The relationships:
(:Organization)-[:HAS]->(:Organization)
(:Rule)-[:MAY_LEAD_TO]->(:Consequence)
(:Rule)-[:APPLIES_TO]->(:Consequence)
(:Rule)-[:APPLIES_TO]->(:Role)
(:Rule)-[:AFFECTS]->(:Role)
(:Rule)-[:AFFECTS]->(:Organization)
(:Rule)-[:AFFECTS]->(:Consequence)
(:Rule)-[:MAY_RESULT_IN]->(:Consequence)
(:Rule)-[:RESTRICTS]->(:Consequence)


In [ ]:
cypher = """
  MATCH (n) 
  RETURN count(n)
  """

In [ ]:
result = kg.query(cypher)
result

[{'count(n)': 46}]

In [31]:
from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

def query_neo4j(query, parameters=None):
    with driver.session() as session:
        result = session.run(query, parameters)
        return result.data()

In [32]:
!pip3 install langchain_openai

In [33]:
import os

import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI
from neo4j import GraphDatabase
from langchain_core.pydantic_v1 import BaseModel, Field


In [34]:
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties: Organization {name: STRING} Role {name:
STRING} Rule {description: STRING, name: STRING} Consequence
{name: STRING} Relationship properties:  The relationships:
(:Organization)-[:HAS]->(:Organization)
(:Rule)-[:MAY_LEAD_TO]->(:Consequence)
(:Rule)-[:APPLIES_TO]->(:Consequence)
(:Rule)-[:APPLIES_TO]->(:Role) (:Rule)-[:AFFECTS]->(:Role)
(:Rule)-[:AFFECTS]->(:Organization)
(:Rule)-[:AFFECTS]->(:Consequence)
(:Rule)-[:MAY_RESULT_IN]->(:Consequence)
(:Rule)-[:RESTRICTS]->(:Consequence)


In [35]:
kg.query("""
    MATCH (rule:Rule {name: 'Late Payment Policy'})-[:MAY_RESULT_IN]->(consequence:Consequence)
    RETURN rule.description, consequence.name
""")

[]

# Study Leave KG


In [ ]:
load_dotenv('.env', override=True)
NEO4J_URI_SL = os.getenv('NEO4J_URI')
NEO4J_USERNAME_SL = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD_SL = os.getenv('NEO4J_PASSWORD_SL')
NEO4J_DATABASE_SL = os.getenv('NEO4J_DATABASE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [ ]:
graph_sl = Neo4jGraph(
    url=NEO4J_URI_SL, username=NEO4J_USERNAME_SL, password=NEO4J_PASSWORD_SL, database=NEO4J_DATABASE_SL
)
graph_sl.refresh_schema()
print(graph_sl.schema)

Node properties:
StudyLeave {budgetaryArrangements: STRING, eligibility: STRING, entitlement: STRING, programDescription: STRING}
StaffMember {title: STRING, department: STRING}
Approver {responsibility: STRING, prerequisite: STRING, title: STRING}
Relationship properties:

The relationships:
(:StaffMember)-[:APPLY_FOR]->(:StudyLeave)
(:Approver)-[:APPROVE]->(:StudyLeave)
(:Approver)-[:REVIEW]->(:StudyLeave)


In [ ]:
# Define the template for generating Cypher queries based on the schema and user questions
CYPHER_GENERATION_TEMPLATE = """Task: Generate Cypher statement to 
query a graph database.
Instructions:
Use only the provided relationship types and properties in the 
schema. Do not use any other relationship types or properties that 
are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than 
for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
# Who are the staff members eligible for study leave?
MATCH (sm:StaffMember)-[:APPLY_FOR]->(sl:StudyLeave)
WHERE toLower(sl.eligibility) CONTAINS 'eligible'
RETURN sm.title, sm.department

# Which approver is responsible for approving study leave?
MATCH (ap:Approver)-[:APPROVE]->(:StudyLeave)
WHERE toLower(ap.responsibility) CONTAINS 'approve'
RETURN ap.title, ap.responsibility

# What responsibilities does the Vice-President have regarding study leave?
MATCH (ap:Approver)
WHERE toLower(ap.title) CONTAINS 'vice-president'
RETURN ap.responsibility

# What are the entitlements for study leave?
MATCH (sl:StudyLeave)
RETURN sl.entitlement

The question is:
{question}"""



In [ ]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

# Initialize the GraphCypherQAChain
# Initialize the GraphCypherQAChain without formatting the template here
qa_chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=graph_sl,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    )
print("QA chain initialized")


QA chain initialized


In [ ]:
def prettyCypherChain(question: str) -> str:
    response = qa_chain.run(question)
    print(textwrap.fill(response, 60))






In [ ]:
prettyCypherChain("What are the entitlements for study leave?")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (sl:StudyLeave)
RETURN sl.entitlement
Full Context:
[{'sl.entitlement': 'Up to 12 months of partially paid leave, depending on the length of service'}]

> Finished chain.
Up to 12 months of partially paid leave, depending on the
length of service.


# Vectored KG

In [ ]:
load_dotenv('.env', override=True)
NEO4J_URI_VK = os.getenv('NEO4J_URI')
NEO4J_USERNAME_VK = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD_VK = os.getenv('NEO4J_PASSWORD_VK')
NEO4J_DATABASE_VK = os.getenv('NEO4J_DATABASE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [ ]:
graph_VK = Neo4jGraph(
    url=NEO4J_URI_VK, username=NEO4J_USERNAME_VK, password=NEO4J_PASSWORD_VK, database=NEO4J_DATABASE_VK
)

The graph retriever starts by identifying relevant entities in the input. For simplicity, we instruct the LLM to identify people, organizations, and actions. To achieve this, we will use LCEL with the newly added with_structured_output method to achieve this.

In [36]:
# Retriever
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")
kg.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the department, students, faculty, staff, actions, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

In [37]:
entity_chain.invoke({"question": "Are individual academic departments authorized to withhold grades from Enrolment Services for any reason?"}).names

InternalServerError: Error code: 500 - {'error': {'message': 'The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_53bd475c0d556cba449d4b57ab40e907 in your email.)', 'type': 'server_error', 'param': None, 'code': None}}

In [38]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph_VK.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [39]:
print(structured_retriever("Are individual academic departments authorized to withhold grades from Enrolment Services for any reason?"))

NameError: name 'graph_VK' is not defined

In [43]:

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=NEO4J_URI_VK,
    username=NEO4J_USERNAME_VK,
    password=NEO4J_PASSWORD_VK,
    search_type="hybrid",
    node_label="node_label",
    text_node_properties=["name"],
    embedding_node_property="embedding"
)
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 72, offset: 71} for query: "UNWIND $data AS row MATCH (n:`node_label`) WHERE elementId(n) = row.id CALL db.create.setVectorProperty(n, 'embedding', row.embedding) YIELD node RETURN count(*)"


## Defining the RAG chain

In [44]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
    ConfigurableField, 
)# Condense a chat history and follow-up question into a standalone question
from langchain_core.output_parsers import StrOutputParser
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [45]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [30]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [46]:
chain.invoke({"question": "Are individual academic departments authorized to withhold grades from Enrolment Services for any reason?"})

Search query: Are individual academic departments authorized to withhold grades from Enrolment Services for any reason?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: MENTIONS)} {position: line: 5, column: 32, offset: 166} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            

'No, individual academic departments are not authorized to withhold grades from Enrolment Services for any reason.'